<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Predicting Wine Quality: Task 3
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading and Exploration

In [6]:
data_main = pd.read_csv("winemag-data-130k-v2.csv")
data = data_main.sample(frac=0.05)
# Remove uninformative columns like "Taster Name" and "Taster Twitter Handle"
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [8]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test = train_test_split(data['description'], data['points'])

In [9]:
print("X_trainval: \t", X_trainval.shape, y_trainval.shape)
print("X_test: \t", X_test.shape, y_test.shape)

X_trainval: 	 (4874,) (4874,)
X_test: 	 (1625,) (1625,)


### BERT Based Model

In [10]:
from transformers import pipeline
nlp = pipeline("feature-extraction")

In [11]:
list_X_trainval = X_trainval.values.tolist()
list_X_test = X_test.values.tolist()
print(X_trainval.shape)
print(X_test.shape)

(4874,)
(1625,)


In [12]:
def nlp_preprocess(dataset):
    nlp_arr = []
    j = 0
    for i in dataset:
        j+=1
        if j%1000 == 0:
            print(j)
        nlp_arr.append(nlp(i))
    return nlp_arr

In [13]:
nlp_X_trainval_preprocess = nlp_preprocess(list_X_trainval)
print("Completed X_trainval preprocessing")

1000
2000
3000
4000
Completed X_trainval preprocessing


In [14]:
%%time
nlp_X_test_preprocess = nlp_preprocess(list_X_test)
print("Completed X_test preprocessing")


1000
Completed X_test preprocessing
CPU times: user 5min 13s, sys: 55.1 s, total: 6min 8s
Wall time: 5min 4s


In [15]:
nlp_X_trainval = pd.DataFrame(nlp_X_trainval_preprocess)
nlp_X_test = pd.DataFrame(nlp_X_test_preprocess)

In [16]:
# Debugging: 
print("X_trainval: \t", nlp_X_trainval.shape)
print("X_test: \t", nlp_X_test.shape)

X_trainval: 	 (4874, 1)
X_test: 	 (1625, 1)


In [49]:
def create_train_test(df):
    dataset_list = np.array(df.values)
    processed = []
    for i in dataset_list:
        processed.append(i[0][0])
    return np.array(processed)

In [50]:
X_trainval_np = create_train_test(nlp_X_trainval)

In [51]:
X_test_np = create_train_test(nlp_X_test)

In [52]:
print(X_test_np.shape)
print(X_trainval_np.shape)

(1625, 768)
(4874, 768)


In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [48]:
%%time
ridge_pipeline = Pipeline(steps=[
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval_np, y_trainval)

print(("BERT Ridge score with GridSearchCV %.2f"
       %grid.score(X_test_np, y_test)))
print("Best param:", grid.best_params_)

BERT Ridge score with GridSearchCV 0.57
Best param: {'regressor__alpha': 1}
CPU times: user 3.42 s, sys: 203 ms, total: 3.62 s
Wall time: 2.26 s


#### Analysis
- Given the significant downsampling of the original dataset, we are impressed by the accuracy of the BERT model. 
- We had to downsample to 5% of the data because of the lack of a GPU and failure to process the entire dataset on the local machine and free Google Collab hardware.